In [ ]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("C:\\Users\\Rudra\\Desktop\\RudraModi_InternshalaResume.pdf")
pdf_data = loader.load()
pdf_data

In [ ]:
groq_api_key = "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"

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
import bs4

loader = WebBaseLoader(
    [
        "https://api.upstox.com/v2/historical-candle/NSE_EQ%7CINE040A01034/day/2024-07-23/2024-07-01",
    ]
    # "https://api.upstox.com/v2/historical-candle/NSE_EQ%7CINE040A01034/day/2024-07-23/2024-07-01",
    # bs_kwargs=dict(parse_only=bs4.SoupStrainer(class_=("question js-question")),
)
web_data = loader.load()
web_data

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
pdf_splitted = text_splitter.split_documents(pdf_data)
web_splitted = text_splitter.split_documents(web_data)
web_splitted

In [ ]:
from langchain_community.embeddings import OllamaEmbeddings

embeddings_obj = OllamaEmbeddings(model="llava-phi3:3.8b")
embeddings = embeddings_obj.embed_documents(pdf_splitted)
embeddings

In [ ]:
from langchain_community.vectorstores import FAISS

db = FAISS.from_documents(pdf_splitted, embeddings_obj)
db

In [ ]:
from langchain_community.vectorstores.chroma import Chroma

db_chroma = Chroma.from_documents(web_splitted, embeddings_obj)
db_chroma

In [ ]:
query = "what is the project for the candidate"
results = db_chroma.similarity_search(query)

In [ ]:
results[0].page_content

In [5]:
from langchain.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_groq import ChatGroq

groq_api_key = "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"
llm = ChatGroq(groq_api_key=groq_api_key, model_name="llama3-70b-8192")
prompt = ChatPromptTemplate.from_template(
    """
Answer the questions based on the provided context only.
Please provide the most accurate response based on the question
<context>
{context}
<context>
Questions:{input}

"""
)
document_chain = create_stuff_documents_chain(llm, prompt)

In [ ]:
from langchain.chains import create_retrieval_chain

retriever = db_chroma.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [ ]:
prompt = "what can be next trend for 5 day onwards from today"

In [ ]:
response = retrieval_chain.invoke(input={"input": prompt, "context": web_splitted})

In [ ]:
print(response["answer"])